In [ ]:
from dotenv import load_dotenv
load_dotenv()

import os
GITHUB_TOKEN = os.environ.get('GITHUB_TOKEN', '')
if not GITHUB_TOKEN:
    raise ValueError("GitHub token not found. Please set GITHUB_TOKEN in your .env file.")

from github import Github

import daft
# Authentication is defined via github.Auth
from github import Auth

# using an access token
auth = Auth.Token(GITHUB_TOKEN)

# Now, we can create a Github instance
github = Github(auth=auth)

: 

In [20]:
@daft.udf(return_dtype=daft.DataType.struct({"name": daft.DataType.string(), "url": daft.DataType.string()}), batch_size=1)
def search_for_repos(query, sort, order, limit):
    query, sort, order, limit = query.to_pylist()[0], sort.to_pylist()[0], order.to_pylist()[0], limit.to_pylist()[0]
    repos = github.search_repositories(query=query, sort=sort, order=order)
    res = []
    for repo in repos[:limit]:
        res.append({"name": repo.name, "url": repo.clone_url})
    return res

In [21]:
import daft

df = daft.from_pydict({"query": ["language:rust"], "sort": ["stars"], "order": ["desc"], "limit": [10]})

df = df.with_column(
    "repo_data",
    search_for_repos(df["query"], df["sort"], df["order"], df["limit"])
)

df.show()


queryUtf8,sortUtf8,orderUtf8,limitInt64,"repo_dataStruct[name: Utf8, url: Utf8]"
language:rust,stars,desc,10,"{name: deno,url: https://github.com/denoland/deno.git,}"
language:rust,stars,desc,10,"{name: rust,url: https://github.com/rust-lang/rust.git,}"
language:rust,stars,desc,10,"{name: tauri,url: https://github.com/tauri-apps/tauri.git,}"
language:rust,stars,desc,10,"{name: rustdesk,url: https://github.com/rustdesk/rustdesk.git,}"
language:rust,stars,desc,10,"{name: sway,url: https://github.com/FuelLabs/sway.git,}"
language:rust,stars,desc,10,"{name: alacritty,url: https://github.com/alacritty/alacritty.git,}"
language:rust,stars,desc,10,"{name: fuel-core,url: https://github.com/FuelLabs/fuel-core.git,}"
language:rust,stars,desc,10,"{name: rustlings,url: https://github.com/rust-lang/rustlings.git,}"
